In [4]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error

In [5]:
df1 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [6]:
df1.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [7]:
df1.shape

(3066766, 19)

In [8]:
df2 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [9]:
df2.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1.0,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142.0,163.0,2.0,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00
1,2.0,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71.0,71.0,4.0,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00
2,2.0,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71.0,71.0,4.0,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00
3,1.0,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132.0,26.0,1.0,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25
4,2.0,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161.0,145.0,1.0,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00


In [10]:
df2.shape

(2913955, 19)

### Duration

In [11]:
df1['duration'] = (df1['tpep_dropoff_datetime'] - df1['tpep_pickup_datetime']) / np.timedelta64(1, 'm')
df2['duration'] = (df2['tpep_dropoff_datetime'] - df2['tpep_pickup_datetime']) / np.timedelta64(1, 'm')

In [12]:
# calculate standard deviation
df1['duration'].std()

42.594351241920904

In [13]:
# remove outliers, keep only duration between 1 - 60 mins
df1_clean = df1[(df1['duration'] >= 1) & (df1['duration'] <= 60)]
len(df1_clean) / len(df1)

0.9812202822125979

In [31]:
# remove outliers, keep only duration between 1 - 60 mins
df2_clean = df2[(df2['duration'] >= 1) & (df2['duration'] <= 60)]
len(df2_clean) / len(df2)

0.9800944077722545

### One-hot encoding

In [14]:
locs = df1_clean[['PULocationID', 'DOLocationID']].astype(str)

In [15]:
dict_locs = locs.to_dict(orient='records')

In [16]:
vec = DictVectorizer()

In [17]:
X = vec.fit_transform(dict_locs)

In [18]:
len(vec.get_feature_names_out())

515

### Model training

In [ ]:
y = df1_clean['duration'].values

In [20]:
lr = LinearRegression()

In [21]:
lr.fit(X, y)

LinearRegression()

In [22]:
y_pred = lr.predict(X)

In [24]:
print(mean_squared_error(y, y_pred, squared=False))

7.649262106350244


### Model testing

In [32]:
locs_test = df2_clean[['PULocationID', 'DOLocationID']].astype(str)
dict_locs_test = locs_test.to_dict(orient='records')
X_test = vec.transform(dict_locs_test)

In [36]:
y_test = df2_clean['duration'].values

In [37]:
y_test_pred = lr.predict(X_test)

In [38]:
print(mean_squared_error(y_test, y_test_pred, squared=False))

13.324389221271757
